In [ ]:
<VSCode.Cell language="markdown">
# Expert Layer Runbook: QCDB Integration
This notebook runs ingestion, embeddings, semantic search, ranking, context building, snapshot, and gap analysis for the Quantum Chemistry + Quantum Physics expert layer.
</VSCode.Cell>
<VSCode.Cell language="python">
# 1) Setup: paths and environment
import os, sys, json, pathlib
ROOT = pathlib.Path(__file__).resolve().parents[2] if '__file__' in globals() else pathlib.Path.cwd().parents[0]
os.environ['QCBD_ROOT'] = str(ROOT)
print('QCBD_ROOT =', os.environ['QCBD_ROOT'])

# Try to make expert package importable
sys.path.append(str(ROOT))
</VSCode.Cell>
<VSCode.Cell language="python">
# 2) Run ingestion pipeline
try:
    from expert.ingestion_pipeline import main as ingest_main
    ingest_main()
    print('[OK] Ingestion pipeline completed.')
except Exception as e:
    print('[ERROR] Ingestion failed:', e)
</VSCode.Cell>
<VSCode.Cell language="python">
# 3) Summarize processed documents
from pathlib import Path
proc_dir = Path(ROOT) / 'expert' / 'sources' / 'processed'
items = list(proc_dir.glob('*.json'))
print(f'Processed docs: {len(items)} found in {proc_dir}')
for p in items[:5]:
    data = json.loads(p.read_text(encoding='utf-8'))
    print('-', p.name, 'quality:', data.get('quality',{}).get('score'))
</VSCode.Cell>
<VSCode.Cell language="python">
# 4) Build embeddings and run semantic searches
try:
    from expert.semantic_search import build_embeddings, search_methods
    embs = build_embeddings(persist=False)
    print('[OK] Built embeddings for', len(embs), 'methods')
    for q in [
        'hartree-fock mean-field',
        'coupled cluster CCSD(T) accuracy',
        'density functional theory benchmarking'
    ]:
        hits = search_methods(q, k=5)
        print(f"\nQuery: {q}")
        for rank, (name, score) in enumerate(hits, 1):
            print(f"  {rank}. {name} (score={score:.3f})")
except Exception as e:
    print('[ERROR] Semantic search failed:', e)
</VSCode.Cell>
<VSCode.Cell language="python">
# 5) Rank methods via query API
try:
    from expert.query_api import rank_methods
    ranked = rank_methods(limit=10)
    print('[OK] Top methods:')
    for i, m in enumerate(ranked[:5], 1):
        print(f"  {i}. {m['name']} :: composite={m.get('composite_score')}")
except Exception as e:
    print('[ERROR] Rank methods failed:', e)
</VSCode.Cell>
<VSCode.Cell language="python">
# 6) Build a context pack for "electron correlation"
try:
    from expert.context_builder import build_context
    ctx = build_context('electron correlation in molecules')
    print('[OK] Context built. Keys:', list(ctx.keys()))
    print('Methods in context:', [m.get('name') for m in ctx.get('methods',[])][:5])
except Exception as e:
    print('[ERROR] Context build failed:', e)
</VSCode.Cell>
<VSCode.Cell language="python">
# 7) Create a graph snapshot
try:
    from expert.versioning import create_snapshot
    snap_path = create_snapshot('runbook_snapshot')
    print('[OK] Snapshot at:', snap_path)
except Exception as e:
    print('[ERROR] Snapshot failed:', e)
</VSCode.Cell>
<VSCode.Cell language="python">
# 8) Gap analysis report
try:
    from expert.gap_analysis import analyze
    report = analyze()
    print('[OK] Gap analysis:')
    print(json.dumps(report, indent=2))
except Exception as e:
    print('[ERROR] Gap analysis failed:', e)
</VSCode.Cell>